### Adaptive Estimator (Single Job Run)
 - Circuit Used - GHZ echo circuit
 - Qubit count - 19 & 63
 - EMST available - Readout EM, DD, ZNE
 - Circuit Transpilation - Default transpiled with Optimization level = 3

In [1]:
import sys
import os

# Add project root (one level up from 'notebooks')
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

In [61]:
# Import requisite libraries
from qiskit import QuantumCircuit
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import efficient_su2
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
from adaptive_error_mitigation import adaptive_estimator
from qiskit_aer.primitives import EstimatorV2 as AerEstimator
import numpy as np

### GHZ Echo Circuit - 19 Qubits

In [68]:
# Define the GHZ-Echo circuit
NUM_QUBITS = 19

ghz = QuantumCircuit(NUM_QUBITS)
ghz.h(0)
for i in range(NUM_QUBITS - 1):
    ghz.cx(i, i + 1)

circuit = QuantumCircuit(NUM_QUBITS)
circuit.compose(ghz, inplace=True)
circuit.barrier()
circuit.compose(ghz.inverse(), inplace=True)
circuit.x(range(NUM_QUBITS))

In [85]:
# Define the backend
service = QiskitRuntimeService()
bknd_markh = service.backend("ibm_marrakesh")

In [86]:
# Transpile the circuit
pm = generate_preset_pass_manager(
    optimization_level=3,
    backend=bknd_markh,
    seed_transpiler=42,
)

isa_ghz = pm.run(circuit)

In [87]:
# Create the logical observable (e.g., "ZZZ")
observable = SparsePauliOp("Z" * NUM_QUBITS)

# Apply the layout from the transpiled circuit
isa_obs = observable.apply_layout(isa_ghz.layout)

In [88]:
pub = (isa_ghz, isa_obs)

#### Ideal Expectation value

In [73]:
aer_estimator = AerEstimator()
job = aer_estimator.run([pub])
evs_ideal = job.result()[0].data.evs
print(f"Ideal estimation value of the GHZ Echo crkt: {evs_ideal}")

Ideal estimation value of the GHZ Echo crkt: -1.0


#### Running using Qiskit default Estimator

In [93]:
estimator = Estimator(bknd_markh)
job = estimator.run([pub])
print(f"Job-Id: {job.job_id()}")

Job-Id: d5s5q1k9u8fs73bd0ia0


In [94]:
job = service.job(job_id="d5s5q1k9u8fs73bd0ia0")
evs_def = job.result()[0].data.evs
print(f"Estimation Value of the GHZ Echo crkt using default qiskit estimator: {evs_def}")

Estimation Value of the GHZ Echo crkt using default qiskit estimator: 0.10021922956467272


#### Running using Adaptive Estimator

In [89]:
# Run the circuit using adaptive estimator
ae_results = adaptive_estimator.run([pub], backend=bknd_markh)


--- Processing Pub 1/1 ---
--- Initiating Adaptive Error Mitigation and Suppression Framework ---

--> DEFAULT SETTING: Using Default Precision set to 0.015625 and default shots 4096

---> HEURISTIC TRIGGERED: Readout Error Threshold Exceeded
     | Metric: MAX READOUT ERROR - 0.0493 (on Qubit 0)
     | Threshold Set: 0.0100 (READOUT_ERROR_THRESHOLD (config.py))
---> ACTION TAKEN: ENABLED Measure Mitigation (TREX)
     | Resilience Level: 1
     | Measure Noise Learning (Randomizations): 32 (NUM_RANDOMIZATIONS (config.py))
---> ACTION TAKEN: ENABLED Measure Twirling
     | Twirling: Measure=True / Gates=False
     | **Derived Parameters:** shots_per_randomization set to 128 (Shots: 4096 (DEFAULT_SHOTS (config.py)) / Randomizations: 32 (NUM_RANDOMIZATIONS (config.py)))

---> Circuit not scheduled. Applying ALAP scheduling...

---> HEURISTIC TRIGGERED: Circuit density Within Range
     | Metric: CIRCUIT DENSITY (CD) - 0.1108
     | Threshold Range: [0.0700, 0.2500] (config.py)
---> ACTI

In [90]:
# Retrieving the set estimator options
est_options = ae_results[0]["est_options"]
est_options

max_execution_time,Unset
default_precision,Unset
default_shots,None
resilience_level,2
seed_estimator,Unset
experimental,Unset
▸environment,EnvironmentOptions
log_level,'WARNING'
callback,None
job_tags,None
private,False


In [91]:
# Retrieving the job_id
job_id = ae_results[0]["job"].job_id()
print(f"Job-Id: {job_id}")

Job-Id: d5s5pgfeglic739v2ing


In [92]:
# Retrieve the estimation value
job = service.job(job_id="d5s5pgfeglic739v2ing")
evs_ae = job.result()[0].data.evs
print(f"Estimation Value of the GHZ Echo crkt using Adaptive Estimator: {evs_ae}")

Estimation Value of the GHZ Echo crkt using Adaptive Estimator: -1.258994913168615


### GHZ Echo Circuit - 63 Qubits

In [95]:
# Define the GHZ-Echo circuit
NUM_QUBITS = 63

ghz = QuantumCircuit(NUM_QUBITS)
ghz.h(0)
for i in range(NUM_QUBITS - 1):
    ghz.cx(i, i + 1)

circuit = QuantumCircuit(NUM_QUBITS)
circuit.compose(ghz, inplace=True)
circuit.barrier()
circuit.compose(ghz.inverse(), inplace=True)
circuit.x(range(NUM_QUBITS))

In [96]:
# Transpile the circuit
pm = generate_preset_pass_manager(
    optimization_level=3,
    backend=bknd_markh,
    seed_transpiler=42,
)

isa_ghz = pm.run(circuit)

In [97]:
# Create the logical observable (e.g., "ZZZ")
observable = SparsePauliOp("Z" * NUM_QUBITS)

# Apply the layout from the transpiled circuit
isa_obs = observable.apply_layout(isa_ghz.layout)

In [98]:
pub = (isa_ghz, isa_obs)

#### Calculating Ideal Expectation value

In [99]:
aer_estimator = AerEstimator()
job = aer_estimator.run([pub])
evs_ideal = job.result()[0].data.evs
print(f"Ideal estimation value of the GHZ Echo crkt: {evs_ideal}")

Ideal estimation value of the GHZ Echo crkt: -1.0


#### Running using Qiskit default Estimator with Resilience level as 2

In [104]:
estimator = Estimator(bknd_markh, options={"resilience_level": 2})
job = estimator.run([pub])
print(f"Job-Id: {job.job_id()}")

Job-Id: d5s5ris9u8fs73bd0jtg


In [105]:
job = service.job(job_id="d5s5ris9u8fs73bd0jtg")
evs_def = job.result()[0].data.evs
print(
    f"Estimation Value of the GHZ Echo crkt using default qiskit estimator with resilience level as 2: {evs_def}"
)

Estimation Value of the GHZ Echo crkt using default qiskit estimator with resilience level as 2: 0.1548651107793773


### Running using Adaptive Estimator

In [100]:
# Run the circuit using adaptive estimator
ae_results = adaptive_estimator.run([pub], backend=bknd_markh)


--- Processing Pub 1/1 ---
--- Initiating Adaptive Error Mitigation and Suppression Framework ---

--> DEFAULT SETTING: Using Default Precision set to 0.015625 and default shots 4096

---> HEURISTIC TRIGGERED: Readout Error Threshold Exceeded
     | Metric: MAX READOUT ERROR - 0.0740 (on Qubit 2)
     | Threshold Set: 0.0100 (READOUT_ERROR_THRESHOLD (config.py))
---> ACTION TAKEN: ENABLED Measure Mitigation (TREX)
     | Resilience Level: 1
     | Measure Noise Learning (Randomizations): 32 (NUM_RANDOMIZATIONS (config.py))
---> ACTION TAKEN: ENABLED Measure Twirling
     | Twirling: Measure=True / Gates=False
     | **Derived Parameters:** shots_per_randomization set to 128 (Shots: 4096 (DEFAULT_SHOTS (config.py)) / Randomizations: 32 (NUM_RANDOMIZATIONS (config.py)))

---> Circuit not scheduled. Applying ALAP scheduling...

Final Estimator Options:
{'_VERSION': 2, 'max_execution_time': Unset, 'environment': EnvironmentOptions(log_level='WARNING', callback=None, job_tags=None, private

In [101]:
# Retrieving the set estimator options
est_options = ae_results[0]["est_options"]
est_options

max_execution_time,Unset
default_precision,Unset
default_shots,None
resilience_level,1
seed_estimator,Unset
experimental,Unset
▸environment,EnvironmentOptions
log_level,'WARNING'
callback,None
job_tags,None
private,False


In [102]:
# Retrieving the job_id
job_id = ae_results[0]["job"].job_id()
print(f"Job-Id: {job_id}")

Job-Id: d5s5r60ubqnc73c4aovg


In [103]:
# Retrieve the estimation value
job = service.job(job_id="d5s5r60ubqnc73c4aovg")
evs_ae = job.result()[0].data.evs
print(f"Estimation Value of the GHZ Echo crkt using Adaptive Estimator: {evs_ae}")

Estimation Value of the GHZ Echo crkt using Adaptive Estimator: 0.06049149338374291


## Performance Analysis: Adaptive Estimator vs. Default Qiskit Estimator

This analysis evaluates the accuracy of the **Adaptive Estimator** compared to the **Default Qiskit Estimator** using the GHZ Echo circuit across two different scales (19 and 63 qubits).

---

### 📊 Summary of Results

| Metric | 19 Qubits | 63 Qubits |
| :--- | :--- | :--- |
| **Ideal Value** | -1.0 | -1.0 |
| **Default Estimator Error** | 1.1002 | 1.1549 |
| **Adaptive Estimator Error** | 0.2590 | 1.0605 |
| **Percentage Improvement** | **76.46%** | **8.17%** |

---

### 🔍 Key Findings

#### 1. 19 Qubits: High Precision Gain
The Adaptive Estimator demonstrates a significant performance leap at this scale. 
* **Observation:** While the default estimator's result was far from the target (yielding an error of **1.1002**), the Adaptive Estimator successfully narrowed this gap to **0.2590**.
* **Impact:** This represents a **76.46% reduction in error**, moving the result much closer to the physical truth.

#### 2. 63 Qubits: Marginal Improvement
At the higher scale of 63 qubits, both estimators encounter challenges due to increased noise and circuit complexity.
* **Observation:** Even with the default estimator utilizing **Resilience Level 2**, the Adaptive Estimator still outperformed it.
* **Impact:** It achieved an **8.17% reduction in error**. While the margin is smaller than the 19-qubit case, it remains the more robust choice for high-qubit counts.

---

### 💡 Conclusion

The **Adaptive Estimator** consistently provides more accurate results than the default Qiskit estimator across both scales. 

* **Scale Efficiency:** It is particularly effective at the 19-qubit level, where it reduces error by over **76%**. 
* **Resilience:** While the improvement margin narrows as the system scales to 63 qubits, the Adaptive Estimator maintains superior accuracy even when compared to the default estimator's higher resilience configurations.

This suggests that for high-fidelity GHZ Echo circuits, the Adaptive Estimator is a critical tool for mitigating estimation drift.